In [1]:
import json
import math
from collections import Counter
from urllib.request import urlopen
import re

from lxml import html
from bs4 import BeautifulSoup
import requests

In [2]:
def getTotalPage(href):
    res = requests.get(href)
    soup = BeautifulSoup(res.text, 'lxml')
    numberReg = re.compile(r'(\d+ )筆')
    mo = numberReg.search(soup.select('ul > li.right')[0].text)
    pageNum = int(mo.group(1))
    totalPages = 150 if math.ceil(pageNum / 20) > 150 else math.ceil(pageNum / 20)
    res.close()
    return totalPages

In [5]:
pagenumber = []
pair = dict()
start= 2007001000
for n in range(1,13):
    start += 1
    herf = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=軟體設計工程師&jobsource=n104bank1&jobcat="+str(start)+"&order=1&page="
    number = getTotalPage(herf)
    print("%d 頁數為 %d"%(start,number))

2007001001 頁數為 40
2007001002 頁數為 8
2007001003 頁數為 69
2007001004 頁數為 150
2007001005 頁數為 89
2007001006 頁數為 150
2007001007 頁數為 59
2007001008 頁數為 27
2007001009 頁數為 39
2007001010 頁數為 9
2007001011 頁數為 4
2007001012 頁數為 37


In [3]:
href = "https://www.104.com.tw/jobbank/joblist/joblist.cfm?keyword=軟體設計工程師&jobsource=n104bank1&jobcat=2007001001&order=1&page="
res = requests.get(href)
soup = BeautifulSoup(res.text, 'lxml')

In [4]:
linkList = []
jobList = soup.select("div.jobname_summary > a")
for job in jobList:
    link ="https://www.104.com.tw"+job["href"]
    linkList.append(link)
linkList

['https://www.104.com.tw/job/?jobno=5jl00&jobsource=104_hotorder',
 'https://www.104.com.tw/job/?jobno=59lb5&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=4z7ww&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=3ha2v&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=4twuk&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=5e76l&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=5h1cp&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=4j446&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=4s3s9&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=4gpqv&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=3mt96&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=51t32&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com.tw/job/?jobno=53dyg&jobsource=n104bank1&hotjob_chr=',
 'https://www.104.com

In [5]:
jobInfo =dict()
link = linkList[3]
res2 = requests.get(link)
soup2 = BeautifulSoup(res2.text, 'lxml')

In [82]:
jobInfo["title"] = soup2.select_one("header > div > h1").contents[0].replace("\r\n","").strip(" ")
jobInfo["company"] = soup2.select_one("a.cn").text
jobInfo["content"] = soup2.select_one("div.content > p").text.replace("\r","").replace(" ","")
jobInfo["tools"] = soup2.select("dd.tool")[0].text
jobInfo["skills"] = soup2.select("dd.tool")[1].text
jobInfo["post_date"] = soup2.select_one("time.update").text.split("：")[1]
jobInfo["other_condition"] = soup2.select("section.info")[1].select("dl > dd ")[-1].text.replace("\r","").replace(" ","")

In [83]:
with open('data.json', 'w') as f:
    json.dump(jobInfo, f)